### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
# Calculate total number of players for Heroes of Pymoli, by screenname
total_players = purchase_data["SN"].nunique()
print(f"Total number of players: {total_players}")

Total number of players: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# Calculate items
### I show 179 unique items instead of 183 like the starter file... confirmed in excel
num_uniq_item = purchase_data["Item Name"].nunique()
avg_price = purchase_data["Price"].mean()
num_purchases = purchase_data["Purchase ID"].count()
total_rev = purchase_data["Price"].sum()

# Create summary dataframe for items
purch_analysis_df = pd.DataFrame({"Number of Unique Items": num_uniq_item, 
                                  "Average Price": avg_price, "Number of Purchases": num_purchases, 
                                  "Total Revenue": total_rev}, index=[0])
purch_analysis_df["Average Price"] = purch_analysis_df["Average Price"].map("${:.2f}".format)
purch_analysis_df["Total Revenue"] = purch_analysis_df["Total Revenue"].map("${:.2f}".format)
purch_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# Create "unique-ified" dataframe based on unique screennames (this is necessary to account for multiple purchases by same SN)
purchase_data_uniq = purchase_data.drop_duplicates(subset=['SN'], keep='first')

In [5]:
# Calculate items
count_male = purchase_data_uniq.loc[purchase_data_uniq["Gender"] == "Male", "Gender"].count()
count_female = purchase_data_uniq.loc[purchase_data_uniq["Gender"] == "Female", "Gender"].count()
count_other = purchase_data_uniq.loc[purchase_data_uniq["Gender"] == "Other / Non-Disclosed", "Gender"].count()
perc_male = purchase_data_uniq.loc[purchase_data_uniq["Gender"] == "Male", "Gender"].count()*100 / total_players
perc_female = purchase_data_uniq.loc[purchase_data_uniq["Gender"] == "Female", "Gender"].count()*100 / total_players
perc_other = purchase_data_uniq.loc[purchase_data_uniq["Gender"] == "Other / Non-Disclosed", "Gender"].count()*100 / total_players

In [6]:
#Create summary dataframe for items
gender_demo_df = pd.DataFrame({"Gender": ["Male", "Female", "Other/Non-Disclosed"], 
                               "Total Count":[count_male, count_female, count_other], 
                               "Percentage of Players":[perc_male, perc_female, perc_other]})

gender_demo_df["Percentage of Players"] = gender_demo_df["Percentage of Players"].map("{:.2f}%".format)
gender_demo_df = gender_demo_df.set_index("Gender")
gender_demo_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# Calculate Items
purchase_count = purchase_data["Purchase ID"].count   

# Set variable for total player count
total_count = count_male + count_female + count_other 

purchase_data["Total Purchase Price"] = purchase_data["Price"]
purchase_data["Average Total Purchase per Person"] = 0

grouped_purchase_data = purchase_data.groupby(["Gender", "SN"])
grouped_purchase_data.sum()

avg_total_purchase_per_person = grouped_purchase_data["Total Purchase Price"].sum()

avg_total_purch_per_person = pd.DataFrame({"Average Total Purchase per Person": avg_total_purchase_per_person})

avg_total_purch_per_person_final = avg_total_purch_per_person.groupby(level=[0]).mean()

purchase_data_pvt = purchase_data.pivot_table(["Purchase ID", "Price", "Total Purchase Price",
                                               "Average Total Purchase per Person"], index=["Gender"],
                                                aggfunc={"Purchase ID": "count", "Price": "mean",
                                                        "Total Purchase Price": "sum"})
# Concat to join both dataframes
frames = [purchase_data_pvt, avg_total_purch_per_person_final]
result = pd.concat(frames, axis=1)

#Prepare summary dataframe
final_purch_analysis = result.rename(columns={"Purchase ID": "Purchase Count", "Price": "Average Purchase Price", 
                                              "Total Purchase Price": "Total Purchase Value",
                                              "Average Total Purchase per Person": "Avg Total Purchase per Person"})
final_purch_analysis = final_purch_analysis[["Purchase Count", "Average Purchase Price", "Total Purchase Value",
                                            "Avg Total Purchase per Person"]]
final_purch_analysis["Average Purchase Price"] = final_purch_analysis["Average Purchase Price"].map("${:.2f}".format)
final_purch_analysis["Total Purchase Value"] = final_purch_analysis["Total Purchase Value"].map("${:.2f}".format)
final_purch_analysis["Avg Total Purchase per Person"] = final_purch_analysis["Avg Total Purchase per Person"].map("${:.2f}".format)
final_purch_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [8]:
# Set up bins

bins = [0, 9, 14, 19, 24, 29, 34, 39, 20000]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data["Age Groups"] = pd.cut(purchase_data["Age"], bins = bins, labels = group_names)
purchase_data.set_index("Age Groups")

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Total Purchase Price,Average Total Purchase per Person
Age Groups,,,,,,,,,
20-24,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,3.53,0
40+,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,1.56,0
20-24,2,Ithergue48,24,Male,92,Final Critic,4.88,4.88,0
20-24,3,Chamassasya86,24,Male,100,Blindscythe,3.27,3.27,0
20-24,4,Iskosia90,23,Male,131,Fury,1.44,1.44,0
20-24,5,Yalae81,22,Male,81,Dreamkiss,3.61,3.61,0
35-39,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18,2.18,0
20-24,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67,2.67,0
20-24,8,Undjask33,22,Male,21,Souleater,1.10,1.10,0


In [9]:
# Calculate Items
purchase_data_grouped = purchase_data.groupby(["Age Groups"])
purchase_data_grouped_unique = purchase_data_grouped.nunique()
purchase_data_grouped_unique = purchase_data_grouped_unique.rename(columns={"SN": "Total Count"})
purchase_data_grouped_unique["Percentage of Players"] = (purchase_data_grouped_unique["Total Count"]/total_count)*100

# Prepare Summary Dataframe
purchase_data_grouped_unique = purchase_data_grouped_unique[["Total Count", "Percentage of Players"]]
purchase_data_grouped_unique["Percentage of Players"] = purchase_data_grouped_unique["Percentage of Players"].map("{:.2f}%".format)
purchase_data_grouped_unique

,Total Count,Percentage of Players
Age Groups,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [10]:
# Calculate purchase count, avg purchase price, and total purchase value
grouped_purchase_data = purchase_data.groupby(["Age Groups"])
average_purchase_price = grouped_purchase_data["Total Purchase Price"].mean()
purchase_count = grouped_purchase_data["Purchase ID"].count()
total_purchase_price = grouped_purchase_data["Total Purchase Price"].sum()

# Calculate the 4th column, Avg total purchase per person 
double_index_grouped_purchase_data = purchase_data.groupby(["Age Groups", "SN"])
avg_total_purchase_per_person = double_index_grouped_purchase_data["Total Purchase Price"].sum()
avg_total_purchase_per_person = pd.DataFrame({"Avg Total Purchase per Person": avg_total_purchase_per_person})
avg_total_purchase_per_person = avg_total_purchase_per_person.groupby(level=[0]).mean()
avg_total_purchase_per_person

# Create a new df for the first three columns, and merge with the df for the 4th column
purchasing_analysis_age_premerge = pd.DataFrame({"Purchase Count": purchase_count, 
                                        "Average Purchase Price": average_purchase_price,
                                        "Total Purchase Value": total_purchase_price})
purchasing_analysis_age = pd.merge(purchasing_analysis_age_premerge, avg_total_purchase_per_person, on="Age Groups")

#Improve formatting of summary table
purchasing_analysis_age["Average Purchase Price"] = purchasing_analysis_age["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis_age["Total Purchase Value"] = purchasing_analysis_age["Total Purchase Value"].map("${:.2f}".format)
purchasing_analysis_age["Avg Total Purchase per Person"] = purchasing_analysis_age["Avg Total Purchase per Person"].map("${:.2f}".format)
purchasing_analysis_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
# create dataframes for the three columns

grouped_purchase_data2 = purchase_data.groupby(["SN"])
total_purchase_price = grouped_purchase_data2["Price"].sum()
total_purchase_price = pd.DataFrame({"Total Purchase Price": total_purchase_price})

grouped_purchase_data2 = purchase_data.groupby(["SN"])
total_purchase_count = grouped_purchase_data2["Purchase ID"].count()
total_purchase_count = pd.DataFrame({"Total Purchase Count": total_purchase_count})

double_index_grouped_purchase_data2 = purchase_data.groupby(["Age Groups", "SN"])
avg_total_purchase_per_person2 = double_index_grouped_purchase_data2["Total Purchase Price"].mean()
avg_total_purchase_per_person2 = pd.DataFrame({"Avg Total Purchase per Person": avg_total_purchase_per_person2})

In [12]:
# Calculate purchase count, avg purchase price, and total purchase value for the top 5 screennames

top_5_total_purchase_price = total_purchase_price["Total Purchase Price"].nlargest(5)

top_5_total_purchase_price_df = pd.DataFrame({"Total Purchase Price": top_5_total_purchase_price})
top_5_total_purchase_price_df["SN"] = top_5_total_purchase_price_df.index

top_5_sn_by_purch_price = top_5_total_purchase_price_df.index.values
top_5_sn_by_purch_price_list = top_5_sn_by_purch_price.tolist()

avg_total_purchase_per_person2.index= avg_total_purchase_per_person2.index.droplevel(0)
avg_total_purchase_per_person2["SN"] = avg_total_purchase_per_person2.index

top_5_avg_purchase_price = avg_total_purchase_per_person2[avg_total_purchase_per_person2["SN"].isin(top_5_sn_by_purch_price_list)]

total_purchase_count["SN"] = total_purchase_count.index
top_5_purchase_count = total_purchase_count[total_purchase_count["SN"].isin(top_5_sn_by_purch_price_list)]

top_5_purchase_count = top_5_purchase_count.drop(columns="SN")
top_5_avg_purchase_price = top_5_avg_purchase_price.drop(columns="SN")
top_5_total_purchase_price_df = top_5_total_purchase_price_df.drop(columns="SN")

In [13]:
# Create summary dataframe
merge1 = pd.merge(top_5_avg_purchase_price, top_5_total_purchase_price_df, on="SN")

In [14]:
merge2 = pd.merge(merge1, top_5_purchase_count, on="SN")
top_spenders_df = merge2

In [15]:
top_spenders_df = top_spenders_df.rename(columns={"Total Purchase Count": "Purchase Count",
                           "Avg Total Purchase per Person": "Average Purchase Price", 
                          "Total Purchase Price": "Total Purchase Value"})
top_spenders_df = top_spenders_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

In [16]:
# Improve formatting of summary table: 
top_spenders_df = top_spenders_df.sort_values(by=["Total Purchase Value"], ascending=False)
top_spenders_df["Average Purchase Price"] = top_spenders_df["Average Purchase Price"].map("${:.2f}".format)
top_spenders_df["Total Purchase Value"] = top_spenders_df["Total Purchase Value"].map("${:.2f}".format)
top_spenders_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [17]:
# Retrieve Data & calculate items
purchase_data_item_select = purchase_data[["Item ID", "Item Name", "Price"]]
purchase_data_item_select = purchase_data_item_select.set_index("Item ID", "Item Name")

purchase_count = purchase_data_item_select.index.value_counts()
purchase_count_df = pd.DataFrame({"Purchase Count": purchase_count})
purchase_count_df

total_purchase_value_items = (purchase_count * purchase_data_item_select["Price"])
total_purchase_value_items_df = purchase_data_item_select.groupby(["Item ID"])["Price"].sum()

In [18]:
# Create summary dataframe
purchase_data_item_select["Purchase Count"] = purchase_count_df
purchase_data_item_select["Total Purchase Value"] = total_purchase_value_items_df

In [19]:
purchase_data_item_select = purchase_data_item_select.drop_duplicates()

In [20]:
purchase_data_item_select = purchase_data_item_select.rename(columns={"Price": "Item Price"})

In [21]:
purchase_data_item_select = purchase_data_item_select[["Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]]

In [22]:
purchase_data_item_select_most_popular = purchase_data_item_select.sort_values(by=["Purchase Count"], ascending = False).head()
purchase_data_item_select_most_popular["Item Price"] = purchase_data_item_select_most_popular["Item Price"].map("${:.2f}".format)
purchase_data_item_select_most_popular["Total Purchase Value"] = purchase_data_item_select_most_popular["Total Purchase Value"].map("${:.2f}".format)
purchase_data_item_select_most_popular
### For some reason, I had a hard time keeping the order here as represented on the example, but it's still sorted by Purchase Count

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [23]:
# Set sort order for table
purchase_data_most_profitable_items = purchase_data_item_select.sort_values(by=["Total Purchase Value"], ascending = False).head()
purchase_data_most_profitable_items["Item Price"] = purchase_data_most_profitable_items["Item Price"].map("${:.2f}".format)
purchase_data_most_profitable_items["Total Purchase Value"] = purchase_data_most_profitable_items["Total Purchase Value"].map("${:.2f}".format)
purchase_data_most_profitable_items

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
